In [1]:
!nvidia-smi

Wed May 11 13:02:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import tensorflow as tf

In [10]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [5]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 99%|███████████████████████████████████████▌| 119M/121M [00:02<00:00, 58.0MB/s]
100%|████████████████████████████████████████| 121M/121M [00:02<00:00, 49.6MB/s]


In [6]:
%cd data

/home/studio-lab-user/My notebooks/Some Deep learning/data


In [ ]:
!unzip ../face-expression-recognition-dataset.zip

In [45]:
img_height=96
img_width=96
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    "images/train", labels='inferred',shuffle=True,image_size=(img_height,
    img_width),seed=123,
 batch_size=128

)
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    "images/validation", labels='inferred',shuffle=True,image_size=(img_height,
    img_width),seed=123,
 batch_size=128

)
classes=train_ds.class_names

Found 28821 files belonging to 7 classes.
Found 7066 files belonging to 7 classes.


In [20]:
classes

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [46]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(img_height,
    img_width,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(len(classes),activation='softmax')
])
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 94, 94, 32)        896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 47, 47, 32)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 45, 45, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 22, 22, 64)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 20, 20, 64)        36928     
                                                                 
 flatten_4 (Flatten)         (None, 25600)           

In [27]:
epochs=10
model.fit(train_ds,epochs=epochs,validation_data=val_ds)

Epoch 1/10
226/226 [==============================] - 7s 17ms/step - loss: 2.7370 - accuracy: 0.3117 - val_loss: 1.6435 - val_accuracy: 0.3664
Epoch 2/10
226/226 [==============================] - 4s 16ms/step - loss: 1.5789 - accuracy: 0.3868 - val_loss: 1.5561 - val_accuracy: 0.4022
Epoch 3/10
226/226 [==============================] - 4s 16ms/step - loss: 1.4890 - accuracy: 0.4249 - val_loss: 1.5199 - val_accuracy: 0.4229
Epoch 4/10
226/226 [==============================] - 4s 15ms/step - loss: 1.4117 - accuracy: 0.4593 - val_loss: 1.4769 - val_accuracy: 0.4377
Epoch 5/10
226/226 [==============================] - 5s 20ms/step - loss: 1.3459 - accuracy: 0.4859 - val_loss: 1.4807 - val_accuracy: 0.4462
Epoch 6/10
226/226 [==============================] - 4s 19ms/step - loss: 1.2769 - accuracy: 0.5167 - val_loss: 1.5220 - val_accuracy: 0.4550
Epoch 7/10
226/226 [==============================] - 5s 20ms/step - loss: 1.2097 - accuracy: 0.5411 - val_loss: 1.5670 - val_accuracy: 0.4527

In [47]:
#pretrained model
pretrained_model=tf.keras.applications.MobileNetV2(input_shape=(img_height,
    img_width,3),include_top=False,weights='imagenet')
pretrained_model.trainable=False

model=tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(classes),activation='softmax')
])
learning_rate=0.001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
model.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d_8   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_18 (Dense)            (None, 7)                 8967      
                                                                 
Total params: 2,266,951
Trainable params: 8,967
Non-trainable params: 2,257,984
_________________________________________________________________


In [48]:
reduced_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=2,
    verbose=1)
callbacks=[reduced_lr]
epochs=20 

model.fit(train_ds,epochs=epochs,validation_data=val_ds,callbacks=callbacks)


Epoch 1/20
226/226 [==============================] - 14s 51ms/step - loss: 1.7689 - accuracy: 0.2838 - val_loss: 1.6965 - val_accuracy: 0.3271 - lr: 0.0010
Epoch 2/20
226/226 [==============================] - 10s 43ms/step - loss: 1.6634 - accuracy: 0.3433 - val_loss: 1.6408 - val_accuracy: 0.3614 - lr: 0.0010
Epoch 3/20
226/226 [==============================] - 10s 44ms/step - loss: 1.6225 - accuracy: 0.3639 - val_loss: 1.6388 - val_accuracy: 0.3626 - lr: 0.0010
Epoch 4/20
226/226 [==============================] - 9s 41ms/step - loss: 1.6015 - accuracy: 0.3755 - val_loss: 1.6101 - val_accuracy: 0.3748 - lr: 0.0010
Epoch 5/20
226/226 [==============================] - 9s 41ms/step - loss: 1.5835 - accuracy: 0.3858 - val_loss: 1.5958 - val_accuracy: 0.3842 - lr: 0.0010
Epoch 6/20
226/226 [==============================] - 9s 41ms/step - loss: 1.5708 - accuracy: 0.3911 - val_loss: 1.5901 - val_accuracy: 0.3864 - lr: 0.0010
Epoch 7/20
226/226 [==============================] - 9s 41ms

Same model with trainable set to true

In [49]:
#pretrained model
pretrained_model=tf.keras.applications.MobileNetV2(input_shape=(img_height,
    img_width,3),include_top=False,weights='imagenet')

model=tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(classes),activation='softmax')
])
learning_rate=0.001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d_9   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_19 (Dense)            (None, 7)                 8967      
                                                                 
Total params: 2,266,951
Trainable params: 2,232,839
Non-trainable params: 34,112
_________________________________________________________________


In [ ]:
reduced_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=2,
    verbose=1)
callbacks=[reduced_lr]
epochs=20 

model.fit(train_ds,epochs=epochs,validation_data=val_ds,callbacks=callbacks)


Efficientnet v2

In [54]:
pretrained_model=tf.keras.applications.efficientnet_v2.EfficientNetV2B0(input_shape=(img_height,img_width,3),include_top=False,weights='imagenet')
pretrained_model.trainable=False

model=tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(classes),activation='softmax')
])
learning_rate=0.001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
model.summary()

24289280/24274472 [==============================] - 0s 0us/step
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Function  (None, 3, 3, 1280)       5919312   
 al)                                                             
                                                                 
 global_average_pooling2d_11  (None, 1280)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_21 (Dense)            (None, 7)                 8967      
                                                                 
Total params: 5,928,279
Trainable params: 8,967
Non-trainable params: 5,919,312
_________________________________________________________________


In [55]:
reduced_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=2,
    verbose=1)
callbacks=[reduced_lr]
epochs=20 

model.fit(train_ds,epochs=epochs,validation_data=val_ds,callbacks=callbacks)


Epoch 1/20
226/226 [==============================] - 19s 60ms/step - loss: 1.5188 - accuracy: 0.4169 - val_loss: 1.3995 - val_accuracy: 0.4710 - lr: 0.0010
Epoch 2/20
226/226 [==============================] - 12s 52ms/step - loss: 1.3594 - accuracy: 0.4863 - val_loss: 1.3406 - val_accuracy: 0.4943 - lr: 0.0010
Epoch 3/20
226/226 [==============================] - 12s 52ms/step - loss: 1.3078 - accuracy: 0.5074 - val_loss: 1.3167 - val_accuracy: 0.5003 - lr: 0.0010
Epoch 4/20
226/226 [==============================] - 12s 52ms/step - loss: 1.2776 - accuracy: 0.5190 - val_loss: 1.3008 - val_accuracy: 0.5083 - lr: 0.0010
Epoch 5/20
226/226 [==============================] - 12s 52ms/step - loss: 1.2552 - accuracy: 0.5299 - val_loss: 1.2878 - val_accuracy: 0.5143 - lr: 0.0010
Epoch 6/20
226/226 [==============================] - 12s 52ms/step - loss: 1.2382 - accuracy: 0.5398 - val_loss: 1.2810 - val_accuracy: 0.5159 - lr: 0.0010
Epoch 7/20
226/226 [==============================] - 12s 

Trying EfficientNetV2B2

In [57]:
pretrained_model=tf.keras.applications.efficientnet_v2.EfficientNetV2B2(input_shape=(img_height,img_width,3),include_top=False,weights='imagenet')
pretrained_model.trainable=False

model=tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(classes),activation='softmax')
])
learning_rate=0.001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
model.summary()

35848192/35839040 [==============================] - 1s 0us/step
Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b2 (Function  (None, 3, 3, 1408)       8769374   
 al)                                                             
                                                                 
 global_average_pooling2d_13  (None, 1408)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_23 (Dense)            (None, 7)                 9863      
                                                                 
Total params: 8,779,237
Trainable params: 9,863
Non-trainable params: 8,769,374
_________________________________________________________________


In [58]:
reduced_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=2,
    verbose=1)
callbacks=[reduced_lr]
epochs=20 

model.fit(train_ds,epochs=epochs,validation_data=val_ds,callbacks=callbacks)


Epoch 1/20
226/226 [==============================] - 25s 80ms/step - loss: 1.5378 - accuracy: 0.4103 - val_loss: 1.4243 - val_accuracy: 0.4612 - lr: 0.0010
Epoch 2/20
226/226 [==============================] - 16s 71ms/step - loss: 1.3789 - accuracy: 0.4819 - val_loss: 1.3610 - val_accuracy: 0.4861 - lr: 0.0010
Epoch 3/20
226/226 [==============================] - 16s 71ms/step - loss: 1.3244 - accuracy: 0.5046 - val_loss: 1.3297 - val_accuracy: 0.4969 - lr: 0.0010
Epoch 4/20
226/226 [==============================] - 16s 71ms/step - loss: 1.2900 - accuracy: 0.5198 - val_loss: 1.3135 - val_accuracy: 0.5034 - lr: 0.0010
Epoch 5/20
226/226 [==============================] - 16s 71ms/step - loss: 1.2669 - accuracy: 0.5283 - val_loss: 1.3015 - val_accuracy: 0.5088 - lr: 0.0010
Epoch 6/20
226/226 [==============================] - 16s 71ms/step - loss: 1.2484 - accuracy: 0.5362 - val_loss: 1.2955 - val_accuracy: 0.5133 - lr: 0.0010
Epoch 7/20
226/226 [==============================] - 16s 

In [2]:
import numpy as np
